run in colab: https://colab.research.google.com/drive/1HSqgNwxLLsNKpAoIMJp1FfHbN-WIW-wU#scrollTo=0xUaMXqXPwlT

only tested on T4 GPU.

cloning yolov5 reporsitory.

In [ ]:
%%bash
# clone yolov5 repository with specific version.
git clone https://github.com/ultralytics/yolov5
cd yolov5
git reset --hard 9cdbd1de6b64193b444365982427d5f6d48d6a97
pip install -r requirements.txt

import utils.

In [ ]:
import sys, os, subprocess
sys.path.append('yolov5')

import torch
import torchvision
import numpy as np
import yaml
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import check_img_size, xywh2xyxy
from yolov5.utils.dataloaders import LoadImages, LoadScreenshots, LoadStreams, LoadImagesAndLabels

import holoviews as hv
%env HV_DOC_HTML=true
hv.extension('bokeh')

from traffic_signs_detection_conformal.general import (process_label, preprocess_image, one_hot_encode, apply_conf_thresh, nms_resize_organize_etc)
from traffic_signs_detection_conformal.general import iou as box_iou
from traffic_signs_detection_conformal.general import YOLO2SKLEARN # sklearn-like interface for yolo. just my preference.
from traffic_signs_detection_conformal.general import adjust_bbox_for_flipud, label_bbox # for plotting.
from traffic_signs_detection_conformal.general import megdown

from traffic_signs_detection_conformal.core import gather_nonconformity_scores, form_prediction_set, matching_iou_hungarian
from traffic_signs_detection_conformal.core_utils import predictions_and_labels, summary_table_and_results

download model and dataset

In [ ]:
dataset_name = 'thah'  # 'thah' (will add 'gtsdb' at some point.)
if dataset_name == 'thah':
  megdown('yolov5.pt', '1sdJcel3M_Y9rkCdk9G8GfYaerln8ciJu', dataset_name) # yolov5 model
  megdown('thah.zip', '1JKmMFoKq5U1wo4_86CfvAuM2N1lLpq55', dataset_name) # thai dataset
  !unzip 'datasets/thah/thah.zip' -d 'datasets/thah/cal'

configure

In [ ]:
# config for pytorch
DEVICE = 'cuda'
DNN = True
DATA = None
half = True
DEVICE = torch.device(DEVICE)

# config for dataset
WEIGHTS = "datasets/thah/yolov5.pt"
path_dataset = 'datasets/thah/cal'
path_cal = os.path.join(path_dataset, "images")
num_classes = 36

# config for yolo
IMGSZ = 640
IOU_THRESH = 0.7
IOU_NMS_THRESH = 0.05
CONF_THRESH = 0.5

# config for conformal prediction
ALPHA_COVERAGE =  0.10
CP_METHOD = 'raps' # lac, aps, raps
k_reg = 5
lambda_reg = 0.01
if CP_METHOD == 'lac': ALPHA_COVERAGE = 1 - ALPHA_COVERAGE

# config for plotting
class_mapping = None # if 'None', show number label.
CAL_SIZE = 100

if 'gtsdb'== dataset_name:
    with open("D:/Year 3/A deep learning approach for Thai traffic sign recognition/train_ultralytics/dataset_settings/GTSDB_splitted_training_set_valsize300.yaml", "r") as f:
        dataset_ = yaml.load(f, Loader=yaml.FullLoader)
    for key, name in dataset_['names'].items():
        name: str
        dataset_['names'][key] = (name
                                .replace(" (prohibitory)", "")
                                .replace(" (other)", "")
                                .replace(" (danger)", "")
                                .replace(" (trucks)", "")
                                .replace(" (mandatory)", "")
                                .replace(" (overtaking)", "")
                                .replace(" (overtaking (trucks))", "")
                                )
    class_mapping = dataset_['names']
elif 'thah'== dataset_name:
    pass

load model and dataset. Split dataset into calibration set and testing set.

In [ ]:
# load model
yolo = DetectMultiBackend(weights=WEIGHTS, device=DEVICE, dnn=DNN, data=DATA, fp16=half).to(DEVICE)
model = YOLO2SKLEARN(yolo)
# load dataset
imgsz = check_img_size(IMGSZ, s=yolo.stride)
dataset = LoadImagesAndLabels(path_cal, img_size=imgsz)

indices = np.arange(len(dataset))
np.random.shuffle(indices)
cal_indices = indices[:CAL_SIZE]
test_indices = indices[CAL_SIZE:]
print("\nCalibration set has {} images".format(CAL_SIZE))
print("Test set has {} images".format(len(dataset) - CAL_SIZE))

Gather nonconformity scores on calibration set.

In [ ]:
nonconformity_scores = []
for idx in cal_indices: # for each image-label pair.
    img, label, path, shape = dataset.__getitem__(idx)

    # process image-label into desired format.
    label = process_label(label)
    img = preprocess_image(img, (IMGSZ, IMGSZ))

    # yolo: making prediction and post-process.
    with torch.no_grad():
        pred = model(img) # forwarding yolo model
    pred = apply_conf_thresh(pred, CONF_THRESH)
    pred = nms_resize_organize_etc(pred, IOU_NMS_THRESH, imgsz)

    # box matching
    iou_matrix = box_iou(label["xyxy"], pred['xyxy'])
    i, j = matching_iou_hungarian(iou_matrix, IOU_THRESH)
    gts_class = label["class_label"][i]
    pred_dist = pred['class_dist'][j]

    if gts_class.shape[0] == 0 or pred_dist.shape[0] == 0:
        continue

    gts_dist = one_hot_encode(gts_class, num_classes=num_classes)
    ncs = gather_nonconformity_scores(gts_dist, pred_dist, CP_METHOD, k_reg, lambda_reg)
    nonconformity_scores.extend(ncs)
nonconformity_scores = np.hstack(nonconformity_scores)

compute and plot quantile of nonconformity scores.

In [ ]:
n = len(nonconformity_scores)
quantile = np.quantile(nonconformity_scores, np.ceil((n + 1) * (1 - ALPHA_COVERAGE)) / n, method="higher")

print("Gathered total of {} nonconformity scores".format(n))

hv.extension('bokeh')
hv_nonconformity_scores = hv.Histogram(np.histogram(nonconformity_scores, bins=50)).opts(width=1000, title='nonconformity scores')
hv_nonconformity_quantile = hv.VLine(quantile).opts(color='red')
hv_overlay = hv_nonconformity_scores * hv_nonconformity_quantile
hv_overlay

make conformalized prediction.

In [ ]:
test_predictions, test_labels = predictions_and_labels(model, dataset, CONF_THRESH, IOU_NMS_THRESH, quantile, CP_METHOD, imgsz, k_reg, lambda_reg, test_indices)

evaluate box-wise coverage

In [ ]:
test_summary_table, test_result = summary_table_and_results(test_labels, test_predictions, IOU_THRESH)
test_result

sample bounding-boxes.

In [ ]:
test_summary_table.sample(10)

visualize output plot.

In [ ]:
idx = np.random.randint(len(dataset))
img, labels, path, shapes = dataset.__getitem__(test_indices[idx])

img = img.permute(1,2,0).to('cpu').numpy()

true_bbox = adjust_bbox_for_flipud(test_labels[idx]['xyxy'] - 0.5, 0)
pred_bbox = adjust_bbox_for_flipud(xywh2xyxy(test_predictions[idx]['xywh']) - 0.5, 0)

true_class = test_labels[idx].get('class_label_mapped', test_labels[idx]['class_label'])
pred_class = test_predictions[idx]['class_label']
pred_class_set = [np.where(ps)[0] for ps in test_predictions[idx]['prediction_set_array']]

print("impath: ", test_predictions[idx]['impath'])
print("target: ", true_class)
print("point prediction: ", pred_class)
print("set prediction:: ", pred_class_set)

hv.extension('bokeh')
hv_img = hv.RGB(img)
hv_true_bbox = label_bbox(hv.Rectangles(true_bbox), true_class.astype(int), 'lightgreen', class_mapping)
hv_pred_bbox = label_bbox(hv.Rectangles(pred_bbox), pred_class.astype(int), 'lightgreen', class_mapping)
hv_pred_set_bbox = label_bbox(hv.Rectangles(pred_bbox), pred_class_set, 'lightgreen', class_mapping)

hv_true = (hv_img * hv_true_bbox).opts(width=450, height=600, title='Ground-truth')
hv_point_pred = (hv_img * hv_pred_bbox).opts(width=450, height=600, title='YOLO\'s base prediction')
hv_set_pred = (hv_img * hv_pred_set_bbox).opts(width=450, height=600, title='Conformalized class prediction')

overlay = hv_true + hv_point_pred + hv_set_pred
overlay.cols(3)